<a href="https://colab.research.google.com/github/marina554/accounting-practice/blob/main/BS%E4%BD%9C%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

# === サンプル仕訳データ ===
data = [
    ["2025-11-01", "現金", 100000, 0],
    ["2025-11-02", "売掛金", 50000, 0],
    ["2025-11-03", "備品", 30000, 0],
    ["2025-11-04", "買掛金", 0, 40000],
    ["2025-11-05", "未払金", 0, 20000],
    ["2025-11-06", "資本金", 0, 120000],
]

df = pd.DataFrame(data, columns=["日付", "勘定科目", "借方金額", "貸方金額"])

# === 残高計算 ===
balance = df.groupby("勘定科目")[["借方金額", "貸方金額"]].sum()
balance["残高"] = balance["借方金額"] - balance["貸方金額"]

# === 自動分類関数 ===
def classify_account(account):
    # 部分一致で自動判定（必要に応じて増やせる）
    if any(k in account for k in ["現金", "預金", "売掛", "備品", "建物", "土地"]):
        return "資産"
    elif any(k in account for k in ["買掛", "未払", "借入"]):
        return "負債"
    elif any(k in account for k in ["資本", "資本金", "利益剰余金"]):
        return "純資産"
    else:
        return "その他"

# === 勘定科目を自動分類 ===
balance["区分"] = balance.index.map(classify_account)

# === 貸借対照表作成 ===
bs_data = balance.groupby("区分")["残高"].sum().to_dict()

# 負債・純資産は貸方残なので符号反転
bs_data["負債"] = -bs_data.get("負債", 0)
bs_data["純資産"] = -bs_data.get("純資産", 0)

# DataFrame化
bs_df = pd.DataFrame(bs_data, index=["金額（円）"]).T

# === 出力 ===
print("=== 貸借対照表（自動分類版） ===")
print(bs_df)
print("\n【資産合計】", bs_df.loc["資産", "金額（円）"])
print("【負債＋純資産】", bs_df.loc["負債", "金額（円）"] + bs_df.loc["純資産", "金額（円）"])

=== 貸借対照表（自動分類版） ===
      金額（円）
純資産  120000
負債    60000
資産   180000

【資産合計】 180000
【負債＋純資産】 180000
